# ReEDS Scenarios on PV ICE Tool USA

To explore different scenarios for furture installation projections of PV (or any technology), ReEDS output data can be useful in providing standard scenarios. ReEDS installation projections are used in this journal as input data to the PV ICE tool. 

Current sections include:

<ol>
    <li> ### Reading a standard ReEDS output file and saving it in a PV ICE input format </li>
<li>### Reading scenarios of interest and running PV ICE tool </li>
<li>###Plotting </li>
<li>### GeoPlotting.</li>
</ol>
    Notes:
   
Scenarios of Interest:
	the Ref.Mod, 
o	95-by-35.Adv, and 
o	95-by-35+Elec.Adv+DR ones


In [1]:
import PV_ICE
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
from IPython.display import display
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 8)

In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP')

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\docs\PV_ICE\TEMP


In [3]:
PV_ICE.__version__

'0+untagged.469.g6e6de9b.dirty'

### Reading REEDS original file to get list of SCENARIOs, PCAs, and STATEs 

In [3]:
reedsFile = str(Path().resolve().parent.parent.parent / 'December Core Scenarios ReEDS Outputs Solar Futures v2a.xlsx')
print ("Input file is stored in %s" % reedsFile)

rawdf = pd.read_excel(reedsFile,
                        sheet_name="UPV Capacity (GW)")
                        #index_col=[0,2,3]) #this casts scenario, PCA and State as levels
#now set year as an index in place
#rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True)
rawdf.set_index(['Scenario','Year','PCA', 'State'], inplace=True)

Input file is stored in C:\Users\sayala\Documents\GitHub\December Core Scenarios ReEDS Outputs Solar Futures v2a.xlsx


In [4]:
scenarios = list(rawdf.index.get_level_values('Scenario').unique())
PCAs = list(rawdf.index.get_level_values('PCA').unique())
STATEs = list(rawdf.index.get_level_values('State').unique())

### Reading GIS inputs

In [5]:
GISfile = str(Path().resolve().parent.parent.parent / 'gis_centroid_n.xlsx')
GIS = pd.read_excel(GISfile)
GIS = GIS.set_index('id')

In [6]:
GIS.head()

,long,lat,country
id,,,
p1,-121.452252,47.820991,USA
p10,-117.159039,35.120104,USA
p100,-78.257714,38.791108,USA
p101,-82.192477,28.708695,USA
p102,-80.563731,26.677092,USA


In [7]:
GIS.loc['p1'].long

-121.4522522

### Create Scenarios in PV_ICE

#### Rename difficult characters from Scenarios Names

In [8]:
simulationname = scenarios
simulationname = [w.replace('+', '_') for w in simulationname]
simulationname

['Reference.Mod',
 'Reference.Adv',
 'Reference.Adv_DR',
 '95-by-35.Mod',
 '95-by-35.Adv',
 '95-by-35.Adv_DR',
 '95-by-35_Elec.Mod',
 '95-by-35_Elec.Adv',
 '95-by-35_Elec.Adv_DR']

#### Downselect to Solar Future scenarios of interest

Scenarios of Interest:
<li> Ref.Mod
<li> 95-by-35.Adv  
<li> 95-by-35+Elec.Adv+DR 

In [9]:
SFscenarios = [simulationname[0], simulationname[4], simulationname[8]]
SFscenarios

['Reference.Mod', '95-by-35.Adv', '95-by-35_Elec.Adv_DR']

#### Create the REFERENCE Scenario and assign Baselines

Keeping track of each scenario as its own PV ICE Object.

In [10]:
#for ii in range (0, 1): #len(scenarios):
i = 0
rr = PV_ICE.Simulation(name='USA', path=testfolder)
for i in range(0, 1):
    filetitle = SFscenarios[i]+'.csv'
    filetitle = os.path.join(testfolder, 'USA', filetitle)    
    rr.createScenario(name=SFscenarios[i], file=filetitle)
    rr.scenario[SFscenarios[i]].addMaterial('glass', file=r'..\baselines\ReedsSubset\baseline_material_glass_Reeds.csv')
    rr.scenario[SFscenarios[i]].addMaterial('silicon', file=r'..\baselines\ReedsSubset\baseline_material_silicon_Reeds.csv')
    rr.scenario[SFscenarios[i]].addMaterial('silver', file=r'..\baselines\ReedsSubset\baseline_material_silver_Reeds.csv')
    rr.scenario[SFscenarios[i]].addMaterial('copper', file=r'..\baselines\ReedsSubset\baseline_material_copper_Reeds.csv')
    rr.scenario[SFscenarios[i]].addMaterial('aluminum', file=r'..\baselines\ReedsSubset\baseline_material_aluminium_Reeds.csv')


path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP


# 2 FINISH: Set characteristics of Recycling to SF values.

#### Calculate Mass Flow

In [11]:
IRENA= False
ELorRL = 'RL'
if IRENA:
    if ELorRL == 'RL':
        weibullInputParams = {'alpha': 5.3759}  # Regular-loss scenario IRENA
    if ELorRL == 'EL':
        weibullInputParams = {'alpha': 2.49}  # Regular-loss scenario IRENA
    rr.calculateMassFlow(weibullInputParams=weibullInputParams, weibullAlphaOnly=True)
    title_Method = 'Irena_'+ELorRL
else:
    rr.calculateMassFlow()
    title_Method = 'PVICE'


Working on Scenario:  Reference.Mod
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  silicon
==> Working on Material :  silver
==> Working on Material :  copper
==> Working on Material :  aluminum


In [12]:
print("Scenarios:", rr.scenario.keys())
print("Module Keys:", rr.scenario[SFscenarios[0]].data.keys())
print("Material Keys: ", rr.scenario[SFscenarios[0]].material['glass'].materialdata.keys())

Scenarios: dict_keys(['Reference.Mod'])
Module Keys: Index(['year', 'new_Installed_Capacity_[MW]', 'mod_eff', 'mod_reliability_t50',
       'mod_reliability_t90', 'mod_degradation', 'mod_lifetime', 'mod_MFG_eff',
       'mod_EOL_collection_eff', 'mod_EOL_collected_recycled',
       'mod_Repowering', 'mod_Repairing', 'Area',
       'Cumulative_Area_disposedby_Failure',
       'Cumulative_Area_disposedby_ProjectLifetime',
       'Cumulative_Area_disposed', 'Cumulative_Active_Area',
       'Installed_Capacity_[W]', 'WeibullParams', 'EOL_on_Year_0',
       'EOL_on_Year_1', 'EOL_on_Year_2', 'EOL_on_Year_3', 'EOL_on_Year_4',
       'EOL_on_Year_5', 'EOL_on_Year_6', 'EOL_on_Year_7', 'EOL_on_Year_8',
       'EOL_on_Year_9', 'EOL_on_Year_10', 'EOL_on_Year_11', 'EOL_on_Year_12',
       'EOL_on_Year_13', 'EOL_on_Year_14', 'EOL_on_Year_15', 'EOL_on_Year_16',
       'EOL_on_Year_17', 'EOL_on_Year_18', 'EOL_on_Year_19', 'EOL_on_Year_20',
       'EOL_on_Year_21', 'EOL_on_Year_22', 'EOL_on_Year_23', '

In [13]:
"""
r1.plotScenariosComparison(keyword='Cumulative_Area_disposedby_Failure')
r1.plotMaterialComparisonAcrossScenarios(material='silicon', keyword='mat_Total_Landfilled')
"""
pass

## Aggregating PCAs Material Landfilled to obtain US totals by Year

In [14]:
keyword='mat_Total_Landfilled'
keyword='mat_Virgin_Stock'


In [15]:
rr.scenario[SFscenarios[0]].material['glass'].materialdata['mat_Total_Landfilled']

0     1.677604e+09
1     1.542129e+09
2     1.864615e+10
3     1.794693e+10
4     5.755228e+10
5     5.626462e+10
6     1.190584e+11
7     1.099151e+11
8     1.465299e+11
9     1.493990e+11
10    2.683464e+11
11    2.427442e+11
12    2.560304e+11
13    2.518589e+11
14    3.347866e+11
15    3.314195e+11
16    5.099833e+11
17    5.069446e+11
18    6.629206e+11
19    6.673354e+11
20    9.351717e+11
21    9.371938e+11
22    9.883536e+11
23    9.976634e+11
24    1.063537e+12
25    1.083313e+12
26    1.167408e+12
27    1.200502e+12
28    1.360140e+12
29    1.403517e+12
30    1.564731e+12
31    1.604215e+12
32    1.736414e+12
33    1.760313e+12
34    1.967942e+12
35    1.991998e+12
36    2.157762e+12
37    2.236637e+12
38    2.591186e+12
39    2.765081e+12
40    3.447321e+12
41    3.559147e+12
Name: mat_Total_Landfilled, dtype: float64

In [18]:
materials = ['glass', 'silicon', 'silver', 'copper', 'aluminum']
keywd = 'mat_Total_Landfilled'

USyearly=pd.DataFrame()

#for jj in range(len(SFscenarios)):
for jj in range(0, 1):
    obj = SFscenarios[jj]
    for ii in range(len(materials)):
        USyearly['Waste_'+materials[ii]+'_'+obj] = rr.scenario[obj].material[materials[ii]].materialdata['mat_Total_Landfilled']

    filter_col = [col for col in USyearly if (col.startswith('Waste') and col.endswith(obj)) ]
    USyearly['Waste_Module_'+obj] = USyearly[filter_col].sum(axis=1)

In [19]:
materials = ['glass', 'silicon', 'silver', 'copper', 'aluminum']
keywd = 'VirginStock_Module_'

#for jj in range(len(SFscenarios)):
for jj in range(0,1):
    obj = SFscenarios[jj]
    for ii in range(len(materials)):
        USyearly['VirginStock_'+materials[ii]+'_'+obj] = rr.scenario[obj].material[materials[ii]].materialdata['mat_Total_Landfilled']

    filter_col = [col for col in USyearly if (col.startswith('Waste') and col.endswith(obj)) ]
    USyearly['VirginStock_Module_'+obj] = USyearly[filter_col].sum(axis=1)

### Converting to grams to METRIC Tons. 


In [20]:
USyearly = USyearly/1000000  # This is the ratio for Metric tonnes
#907185 -- this is for US tons


### Adding Installed Capacity to US

In [23]:
keyword='Installed_Capacity_[W]'

#for jj in range(len(SFscenarios)):
for jj in range(0, 1):
    obj = SFscenarios[jj]
    USyearly["Capacity_"+obj] = rr.scenario[obj].data[keyword]
 

In [24]:
USyearly.head(20)

,Waste_glass_Reference.Mod,Waste_silicon_Reference.Mod,Waste_silver_Reference.Mod,Waste_copper_Reference.Mod,Waste_aluminum_Reference.Mod,Waste_Module_Reference.Mod,VirginStock_glass_Reference.Mod,VirginStock_silicon_Reference.Mod,VirginStock_silver_Reference.Mod,VirginStock_copper_Reference.Mod,VirginStock_aluminum_Reference.Mod,VirginStock_Module_Reference.Mod,Capacity_Reference.Mod
0,1677.604029,645.569495,4.493061,0.419352,0.000000e+00,2328.085937,1677.604029,645.569495,4.493061,0.419352,0.000000e+00,4.656172e+03,1.039020e+08
1,1542.129396,294.232988,2.106274,0.408244,3.034419e-07,1838.876902,1542.129396,294.232988,2.106274,0.408244,3.034419e-07,3.677754e+03,2.070834e+08
2,18646.154082,3511.463031,21.222768,4.936146,2.942192e-05,22183.776056,18646.154082,3511.463031,21.222768,4.936146,2.942192e-05,4.436755e+04,1.486954e+09
3,17946.926697,3521.239799,16.341542,4.751042,4.524681e-04,21489.259533,17946.926697,3521.239799,16.341542,4.751042,4.524681e-04,4.297852e+04,2.758626e+09
4,57552.283109,11158.429476,38.159242,15.848832,3.548157e-03,68764.724207,57552.283109,11158.429476,38.159242,15.848832,3.548157e-03,1.375294e+05,7.097585e+09
5,56264.623717,10608.341226,33.974876,15.196321,1.984699e-02,66922.155988,56264.623717,10608.341226,33.974876,15.196321,1.984699e-02,1.338443e+05,1.141307e+10
6,119058.391060,23596.223473,64.995132,36.990533,8.563095e-02,142756.685829,119058.391060,23596.223473,64.995132,36.990533,8.563095e-02,2.855134e+05,2.148921e+10
7,109915.057687,20789.522414,58.288941,39.177919,2.989630e-01,130802.345924,109915.057687,20789.522414,58.288941,39.177919,2.989630e-01,2.616047e+05,3.151968e+10
8,146529.920393,23328.102403,82.772508,64.962707,8.845051e-01,170006.642516,146529.920393,23328.102403,82.772508,64.962707,8.845051e-01,3.400133e+05,4.641201e+10
9,149399.000328,18288.276839,83.887837,64.431714,2.303586e+00,167837.900304,149399.000328,18288.276839,83.887837,64.431714,2.303586e+00,3.356758e+05,6.124645e+10
